# Watershed distance transform for 3-dimensional data

In [ ]:
import os
import errno
import argparse

import numpy as np
from tensorflow.python.keras.optimizers import SGD, Adam
from tensorflow.python.keras import backend as K

import deepcell as dc
from deepcell import get_data
from deepcell import make_training_data
from deepcell import rate_scheduler
from deepcell.model_zoo import bn_feature_net_3D
from deepcell.model_zoo import bn_feature_net_11x61x61_3D
from deepcell.model_zoo import dilated_bn_feature_net_11x61x61_3D
from deepcell.training import train_model_movie
from deepcell.training import train_model_sample_movie
from deepcell.training import train_model_watershed_3D

## Conv Based Training

In [ ]:
DATA_OUTPUT_MODE = 'conv'
BORDER_MODE = 'same'
RESIZE = False
RESHAPE_SIZE = 256
NUM_FRAMES = 30  # get first N frames from each training folder
BINS = 4  # number of distance bins to classify

# filepath constants
DATA_DIR = '/data/data'
MODEL_DIR = '/data/models'
NPZ_DIR = '/data/npz_data'
RESULTS_DIR = '/data/results'
EXPORT_DIR = '/data/exports'
PREFIX = 'cells/MouseBrain/generic'
CONV_DATA_FILE = 'MouseBrain_3d_watershed_{}'.format(DATA_OUTPUT_MODE)

# Check for channels_first or channels_last
IS_CHANNELS_FIRST = K.image_data_format() == 'channels_first'

ROW_AXIS = 3 if IS_CHANNELS_FIRST else 2
COL_AXIS = 4 if IS_CHANNELS_FIRST else 3
CHANNEL_AXIS = 1 if IS_CHANNELS_FIRST else 4

# create these directories if they do not exist
for d in (NPZ_DIR, MODEL_DIR, RESULTS_DIR):
    try:
        os.makedirs(os.path.join(d, PREFIX))
        print('Created new directory:', os.path.join(d, PREFIX))
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

In [ ]:
# Create the training data
make_training_data(
    dimensionality=3,
    direc_name=os.path.join(DATA_DIR, PREFIX),
    file_name_save=os.path.join(NPZ_DIR, PREFIX, CONV_DATA_FILE),
    channel_names=['slice'],  # for iterating over stacks of images from a montage
    training_direcs=['set6'],
    output_mode=DATA_OUTPUT_MODE,
    window_size_x=30,
    window_size_y=30,
    window_size_z=3,
    border_mode=BORDER_MODE,
    reshape_size=None if not RESIZE else RESHAPE_SIZE,
    process=True,
    process_std=True,
    display=False,
    num_frames=NUM_FRAMES,
    num_of_frames_to_display=5,
    verbose=True,
    montage_mode=True,  # annotation folder has montaged sub-dirs
    annotation_name='',  # basically channel name but for annotated images
    raw_image_direc='stacked_raw',
    annotation_direc='annotated/all_montages')

if os.path.isfile(os.path.join(NPZ_DIR, PREFIX, CONV_DATA_FILE) + '.npz'):
    print('\nData saved to', os.path.join(NPZ_DIR, PREFIX, CONV_DATA_FILE) + '.npz')
else:
    raise Exception('Uh Oh!  Your data file did not save properly :(')

In [ ]:
# Load the training data from NPZ into a numpy array
training_data = np.load(os.path.join(NPZ_DIR, PREFIX, CONV_DATA_FILE + '.npz'))

X, y = training_data['X'], training_data['y']
print('X.shape: {}\ny.shape: {}'.format(X.shape, y.shape))

# Set up training parameters
n_epoch = 16
batch_size = 1
frames_per_batch = 10
optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# save the size of the input data for batch_shape model parameter
size = (RESHAPE_SIZE, RESHAPE_SIZE) if RESIZE else X.shape[ROW_AXIS:COL_AXIS + 1]
if IS_CHANNELS_FIRST:
    batch_shape = (batch_size, X.shape[CHANNEL_AXIS], frames_per_batch, size[0], size[1])
else:
    batch_shape = (batch_size, frames_per_batch, size[0], size[1], X.shape[CHANNEL_AXIS])

In [ ]:
# First, create a foreground/background separation model

# Instantiate the model
fgbg_model = bn_feature_net_3D(
    n_features=2,  # segmentation mask (is_cell, is_not_cell)
    batch_shape=batch_shape,
    norm_method='whole_image')

# Train the model
train_model_movie(
    model=fgbg_model,
    expt='fgbg',
    dataset=CONV_DATA_FILE,
    optimizer=optimizer,
    batch_size=batch_size,
    number_of_frames=frames_per_batch,
    n_epoch=n_epoch,
    direc_save=os.path.join(MODEL_DIR, PREFIX),
    direc_data=os.path.join(NPZ_DIR, PREFIX),
    lr_sched=rate_scheduler(lr=0.01, decay=0.95),
    rotation_range=180,
    flip=True,
    shear=0)

In [ ]:
# Next, Create a model for the watershed energy transform

# Instantiate the model
watershed_model = bn_feature_net_3D(
    n_features=BINS,
    batch_shape=batch_shape,
    norm_method='whole_image')

# Train the model
train_model_watershed_3D(
    model=watershed_model,
    expt='watershed',
    dataset=CONV_DATA_FILE,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epoch=n_epoch,
    distance_bins=BINS,
    direc_save=os.path.join(MODEL_DIR, PREFIX),
    direc_data=os.path.join(NPZ_DIR, PREFIX),
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False)

In [ ]:
# With the trained model, make predictions on testing data
training_data_file = os.path.join(NPZ_DIR, PREFIX, CONV_DATA_FILE + '.npz')

train_dict, (X_test, y_test) = get_data(
    training_data_file, mode=DATA_OUTPUT_MODE, seed=21)

# To predict on variable number of frames
# we need to re-instantiate model and load weights
model_fn = bn_feature_net_3D
watershed_weights_file = '2018-07-15_MouseBrain_3d_watershed_conv_watershed_0.h5'
watershed_weights_file = os.path.join(MODEL_DIR, PREFIX, watershed_weights_file)

fg_bg_weights_file = '2018-07-15_MouseBrain_3d_watershed_conv_fgbg_0.h5'
fg_bg_weights_file = os.path.join(MODEL_DIR, PREFIX, fg_bg_weights_file)

fgbg_model = model_fn(n_features=2, batch_shape=X_test.shape, norm_method='whole_image')
fgbg_model.load_weights(fg_bg_weights_file)

watershed_model = model_fn(n_features=BINS, batch_shape=X_test.shape, norm_method='whole_image')
watershed_model.load_weights(watershed_weights_file)

In [ ]:
test_images = watershed_model.predict(X_test)
test_images_fgbg = fgbg_model.predict(X_test)

print('watershed transform shape:', test_images.shape)
print('segmentation mask shape:', test_images_fgbg.shape)

argmax_images = []
for i in range(test_images.shape[0]):
    max_image = np.argmax(test_images[i], axis=CHANNEL_AXIS - 1)
    argmax_images.append(max_image)
argmax_images = np.array(argmax_images)
argmax_images = np.expand_dims(argmax_images, axis=CHANNEL_AXIS)

print('watershed argmax shape:', argmax_images.shape)

In [ ]:
# threshold the foreground/background
# and remove back ground from watershed transform
threshold = 0.8
if IS_CHANNELS_FIRST:
    fg_thresh = test_images_fgbg[:, 1, :, :, :] > threshold
else:
    fg_thresh = test_images_fgbg[:, :, :, :, 1] > threshold

fg_thresh = np.expand_dims(fg_thresh.astype(np.int16), axis=CHANNEL_AXIS)
argmax_images_post_fgbg = argmax_images * fg_thresh

In [ ]:
# Apply watershed method with the distance transform as seed
from scipy import ndimage
from skimage.morphology import watershed
from skimage.feature import peak_local_max

watershed_images = []
for i in range(argmax_images_post_fgbg.shape[0]):
    if IS_CHANNELS_FIRST:
        image = fg_thresh[i, 0, :, :, :]
        distance = argmax_images_post_fgbg[i, 0, :, :, :]
    else:
        image = fg_thresh[i, :, :, :, 0]
        distance = argmax_images_post_fgbg[i, :, :, :, 0]

    local_maxi = peak_local_max(distance, min_distance=10, indices=False, labels=image)

    markers = ndimage.label(local_maxi)[0]
    segments = watershed(-distance, markers, mask=image)
    watershed_images.append(segments)

watershed_images = np.array(watershed_images)
watershed_images = np.expand_dims(watershed_images, axis=CHANNEL_AXIS)

In [ ]:
# Plot the results
import matplotlib.pyplot as plt
import matplotlib.animation as animation

index = 0
frame = 19

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(X_test[index, frame, :, :, 0])
ax[0].set_title('Source Image')

ax[1].imshow(test_images_fgbg[index, frame, :, :, 1])
ax[1].set_title('Segmentation Prediction')

ax[2].imshow(fg_thresh[index, frame, :, :, 0], cmap='jet')
ax[2].set_title('Thresholded Segmentation')

ax[3].imshow(argmax_images[index, frame, :, :, 0], cmap='jet')
ax[3].set_title('Watershed Transform')

ax[4].imshow(argmax_images_post_fgbg[index, frame, :, :, 0], cmap='jet')
ax[4].set_title('Watershed Transform w/o Background')

ax[5].imshow(watershed_images[index, frame, :, :, 0], cmap='jet')
ax[5].set_title('Watershed Segmentation')

fig.tight_layout()
plt.show()

# Sample Based Training

In [ ]:
DATA_OUTPUT_MODE = 'sample'
BORDER_MODE = 'valid'
RESIZE = False
RESHAPE_SIZE = 256
NUM_FRAMES = 30
BINS = 4

# filepath constants
DATA_DIR = '/data/data'
MODEL_DIR = '/data/models'
NPZ_DIR = '/data/npz_data'
RESULTS_DIR = '/data/results'
EXPORT_DIR = '/data/exports'
PREFIX = 'cells/MouseBrain/generic'
WATERSHED_DATA_FILE = 'MouseBrain_3d_watershed_{}'.format(DATA_OUTPUT_MODE)
FG_BG_DATA_FILE = 'MouseBrain_3d_{}'.format(DATA_OUTPUT_MODE)

# Check for channels_first or channels_last
IS_CHANNELS_FIRST = K.image_data_format() == 'channels_first'

ROW_AXIS = 3 if IS_CHANNELS_FIRST else 2
COL_AXIS = 4 if IS_CHANNELS_FIRST else 3
CHANNEL_AXIS = 1 if IS_CHANNELS_FIRST else 4

In [ ]:
# Make the training data for foreground/background segmentation
make_training_data(
    dimensionality=3,
    direc_name=os.path.join(DATA_DIR, PREFIX),
    file_name_save=os.path.join(NPZ_DIR, PREFIX, FG_BG_DATA_FILE),
    channel_names=['slice'],  # for iterating over stacks of images from a montage
    training_direcs=['set6'],
    output_mode=DATA_OUTPUT_MODE,
    max_training_examples=1e5,
    window_size_x=30,
    window_size_y=30,
    window_size_z=3,
    border_mode=BORDER_MODE,
    reshape_size=None if not RESIZE else RESHAPE_SIZE,
    distance_transform=False,
    distance_bins=4,
    display=False,
    num_frames=NUM_FRAMES,
    num_of_frames_to_display=5,
    verbose=False,
    montage_mode=True,  # annotation folder has montaged sub-dirs
    annotation_name='',  # basically channel name but for annotated images
    raw_image_direc='stacked_raw',
    annotation_direc='annotated/all_montages')

if os.path.isfile(os.path.join(NPZ_DIR, PREFIX, FG_BG_DATA_FILE) + '.npz'):
    print('Data Saved to', os.path.join(NPZ_DIR, PREFIX, FG_BG_DATA_FILE) + '.npz')
else:
    raise Exception('Uh Oh!  Your data file did not save properly :(')

In [ ]:
# Make the training data for foreground/background segmentation
make_training_data(
    dimensionality=3,
    direc_name=os.path.join(DATA_DIR, PREFIX),
    file_name_save=os.path.join(NPZ_DIR, PREFIX, WATERSHED_DATA_FILE),
    channel_names=['slice'],  # for iterating over stacks of images from a montage
    training_direcs=['set6'],
    output_mode=DATA_OUTPUT_MODE,
    max_training_examples=1e5,
    window_size_x=30,
    window_size_y=30,
    window_size_z=3,
    border_mode=BORDER_MODE,
    reshape_size=None if not RESIZE else RESHAPE_SIZE,
    distance_transform=True,
    distance_bins=4,
    display=False,
    num_frames=NUM_FRAMES,
    num_of_frames_to_display=5,
    verbose=False,
    montage_mode=True,  # annotation folder has montaged sub-dirs
    annotation_name='',  # basically channel name but for annotated images
    raw_image_direc='stacked_raw',
    annotation_direc='annotated/all_montages')

if os.path.isfile(os.path.join(NPZ_DIR, PREFIX, WATERSHED_DATA_FILE) + '.npz'):
    print('Data Saved to', os.path.join(NPZ_DIR, PREFIX, WATERSHED_DATA_FILE) + '.npz')
else:
    raise Exception('Uh Oh!  Your data file did not save properly :(')

In [ ]:
# Load the training data from NPZ into a numpy array
training_data = np.load(os.path.join(NPZ_DIR, PREFIX, FG_BG_DATA_FILE + '.npz'))

X, y = training_data['X'], training_data['y']
print('X.shape: {}\ny.shape: {}'.format(X.shape, y.shape))

# Set up training parameters
n_epoch = 16
batch_size = 32
frames_per_batch = 10
optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
lr_sched = rate_scheduler(lr=0.01, decay=0.99)

In [ ]:
# First, create a foreground/background separation model

# Instantiate the model
fgbg_model = bn_feature_net_11x61x61_3D(
    n_features=2,
    n_channels=X.shape[CHANNEL_AXIS])

# Train the model
train_model_sample_movie(
    model=fgbg_model,
    dataset=FG_BG_DATA_FILE,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epoch=n_epoch,
    direc_save=os.path.join(MODEL_DIR, PREFIX),
    direc_data=os.path.join(NPZ_DIR, PREFIX),
    expt='fgbg',
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False)

In [ ]:
# Load the training data from NPZ into a numpy array
training_data = np.load(os.path.join(NPZ_DIR, PREFIX, WATERSHED_DATA_FILE + '.npz'))

X, y = training_data['X'], training_data['y']
print('X.shape: {}\ny.shape: {}'.format(X.shape, y.shape))

# Set up training parameters
n_epoch = 16
batch_size = 32
frames_per_batch = 10
optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
lr_sched = rate_scheduler(lr=0.01, decay=0.99)

In [ ]:
# Next, Create a model for the watershed energy transform

# Instantiate the model
watershed_model = bn_feature_net_11x61x61_3D(
    n_features=BINS,
    n_channels=X.shape[CHANNEL_AXIS])

# Train the model
train_model_sample_movie(
    model=watershed_model,
    dataset=WATERSHED_DATA_FILE,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epoch=n_epoch,
    direc_save=os.path.join(MODEL_DIR, PREFIX),
    direc_data=os.path.join(NPZ_DIR, PREFIX),
    expt='watershed',
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False)

In [ ]:
# With the trained models, make predictions on testing data
training_data_file = os.path.join(NPZ_DIR, PREFIX, CONV_DATA_FILE + '.npz')

train_dict, (X_test, y_test) = get_data(
    training_data_file, mode='conv', seed=21)

# Re-initializing dilated models for sample-mode predictions
model_fn = dilated_bn_feature_net_11x61x61_3D

watershed_weights_file = '2018-07-16_MouseBrain_3d_watershed_sample_watershed_0.h5'
watershed_weights_file = os.path.join(MODEL_DIR, PREFIX, watershed_weights_file)

fg_bg_weights_file = '2018-07-15_MouseBrain_3d_sample_fgbg_0.h5'
fg_bg_weights_file = os.path.join(MODEL_DIR, PREFIX, fg_bg_weights_file)

run_watershed_model = dilated_bn_feature_net_11x61x61_3D(
    n_features=BINS,
    input_shape=X_test.shape[1:])
run_watershed_model.load_weights(watershed_weights_file, by_name=True)

run_fgbg_model = dilated_bn_feature_net_11x61x61_3D(
    n_features=2,
    input_shape=X_test.shape[1:])
run_fgbg_model.load_weights(fg_bg_weights_file, by_name=True)

In [ ]:
# With the trained models, make predictions on testing data
test_images = run_watershed_model.predict(X_test)
test_images_fgbg = run_fgbg_model.predict(X_test)

print('watershed transform shape:', test_images.shape)
print('segmentation mask shape:', test_images_fgbg.shape)

argmax_images = []
for i in range(test_images.shape[0]):
    max_image = np.argmax(test_images[i], axis=CHANNEL_AXIS - 1)
    argmax_images.append(max_image)
argmax_images = np.array(argmax_images)
argmax_images = np.expand_dims(argmax_images, axis=CHANNEL_AXIS)

print('watershed argmax shape:', argmax_images.shape)

In [ ]:
# threshold the foreground/background
# and remove back ground from watershed transform
threshold = 0.8
if IS_CHANNELS_FIRST:
    fg_thresh = test_images_fgbg[:, 1, :, :, :] > threshold
else:
    fg_thresh = test_images_fgbg[:, :, :, :, 1] > threshold

fg_thresh = np.expand_dims(fg_thresh.astype(np.int16), axis=CHANNEL_AXIS)
argmax_images_post_fgbg = argmax_images * fg_thresh

In [ ]:
# Apply watershed method with the distance transform as seed
from scipy import ndimage
from skimage.morphology import watershed
from skimage.feature import peak_local_max

watershed_images = []
for i in range(argmax_images_post_fgbg.shape[0]):
    if IS_CHANNELS_FIRST:
        image = fg_thresh[i, 0, :, :, :]
        distance = argmax_images_post_fgbg[i, 0, :, :, :]
    else:
        image = fg_thresh[i, :, :, :, 0]
        distance = argmax_images_post_fgbg[i, :, :, :, 0]

    local_maxi = peak_local_max(distance, min_distance=10, indices=False, labels=image)

    markers = ndimage.label(local_maxi)[0]
    segments = watershed(-distance, markers, mask=image)
    watershed_images.append(segments)

watershed_images = np.array(watershed_images)
watershed_images = np.expand_dims(watershed_images, axis=CHANNEL_AXIS)

In [ ]:
# Plot the results
import matplotlib.pyplot as plt
import matplotlib.animation as animation

index = 1
frame = 14

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(X_test[index, frame, :, :, 0])
ax[0].set_title('Source Image')

ax[1].imshow(test_images_fgbg[index, frame, :, :, 1])
ax[1].set_title('Segmentation Prediction')

ax[2].imshow(fg_thresh[index, frame, :, :, 0], cmap='jet')
ax[2].set_title('Thresholded Segmentation')

ax[3].imshow(argmax_images[index, frame, :, :, 0], cmap='jet')
ax[3].set_title('Watershed Transform')

ax[4].imshow(argmax_images_post_fgbg[index, frame, :, :, 0], cmap='jet')
ax[4].set_title('Watershed Transform w/o Background')

ax[5].imshow(watershed_images[index, frame, :, :, 0], cmap='jet')
ax[5].set_title('Watershed Segmentation')

fig.tight_layout()
plt.show()